In [1]:
"""
@author: mdn769@usask.ca

'''
This will first generate the commit pattern files after reading the XML file generated by srcML tool. 
Then it will convert those commit patterns to GML graph representation. 
'''
"""
import xml.etree.ElementTree as ET
from IPython.display import clear_output
import os

import networkx as nx
#import matplotlib.pyplot as plt

from collections import Counter
import pandas as pd

In [2]:
G = nx.MultiDiGraph()
patterns=[]
def print_path(n, val=""):   
    str_val=str(n.tag).replace("{http://www.srcML.org/srcML/src}","").replace("{http://www.srcML.org/srcML/cpp}","")
    val+="-"+str_val
    length=len(n)
    #print(length, str_val)
    if(length>0):
        for x in n:
            print_path(x, val)
    elif(len(val)>0): 
        newVal = val.replace("decl_stmt-decl-", "decl-").replace(
            "argument_list-argument-", "argument-").replace(
            "expr_stmt-expr-", "expr-").replace(
            "parameter_list-parameter-", "parameter-").replace(
            "block-block_content-", "block-").replace(
            "if_stmt-if-", "if-").replace(
            "-empty_stmt", "")
        
        patterns.append(newVal) #remove unwanted patterns
        
        
def generate_graph(patterns):
    G.clear()
    node_list=[]    
    for cp in patterns:
        path=cp[1:].split('-') #First position has unnecessary '-'
        for n in path:
            if(n not in node_list):
                G.add_node(n) #Add unique nodes. 
        for i in range(0, len(path)-1):
            #print(path[i], "-->", path[i+1])
            if(G.has_edge(path[i], path[i+1]) == False):
                G.add_edge(path[i], path[i+1])     
    return G


In [3]:
file_path = "C:/Users/mdn769/BugFixStatistics/SCGrunable/runable/"

In [12]:
''' This will find a list of unique token sequences (TS Features) from all the files in this subject system. Each item of the list will be
the feature name of the dataset to be prepared. 
'''
list_ts_features = []
count = 0
inPath = file_path+ "source_xml/qtbase/added/"
totalFiles = len(os.listdir(inPath))
for file in os.listdir(inPath):   
#     if(count == 5):
#         break
    #print(inPath+file)
    file_tokens = []
    if(file[-4:]==".xml"):
        clear_output(wait=True)
        print("Working for: ("+str(count)+"/"+str(totalFiles)+")" + ", "+ file)    
        try:
            tree = ET.parse(inPath+file)
            root = tree.getroot()
            patterns=[]
            for n in root:
                print_path(n)
            
            for p in patterns:
                for t in p.split('-'):
                    if(len(t)>0):
                        file_tokens.append(t)
                        
            for ngram in [2, 3]: #n-gram = 2 & 3
                temp=zip(*[file_tokens[i:] for i in range(0, ngram)]) 
                ans=[' '.join(ngram) for ngram in temp]
            
            for a in ans:
                if(a not in list_ts_features):
                    list_ts_features.append(a)
                    
        except: not_worked += ss+", "+ctype+", "+file+"\n"

    count+=1
    


Working for: (2604/2605), ffef22cae224fd6c575e64a73e5017e5855c9aff.cpp.xml


In [14]:
print(len(list_ts_features), list_ts_features[0:10])

6461 ['decl type specifier', 'type specifier decl', 'specifier decl type', 'decl type name', 'type name decl', 'name decl name', 'decl name decl', 'name decl init', 'decl init expr', 'init expr call']


In [26]:
#Counter(list_ts_features)

In [15]:
''' Prepare the dataset. For each commit, list_pattern is the list of feature values. 
'''
dataset = [["commit_id"] + list_ts_features] # prepare the top row of the dataset. 
#subject_systems=["bitcoin", "jenkins", "litecoin", "lucene", "mongo", "oozie"]
subject_systems=["qtbase"]
#commit_type=["added", "deleted"]
commit_type=["added"]
not_worked=""
# fileContent = open("ssNames.txt", "r", encoding='utf-8').read()
# #print("Length of File Content", len(fileContent))
# ssNames = fileContent.split('\n')

for ss in subject_systems:
    if(len(ss.strip()) == 0): #Ignore the last empty line....
        break
    for ctype in commit_type:
        inPath = file_path+ "source_xml/"+ss+"/"+ctype+"/"
        #outPath = "xml_gml/"+ss+"/"+ctype+"/"
        
        count = 0
        totalFiles = len(os.listdir(inPath))
        for file in os.listdir(inPath):   
            #if(count == 10):
                #break
            #print(inPath+file)
            if(file[-4:]==".xml"):
                clear_output(wait=True)
                print("Working for: ("+str(count)+"/"+str(totalFiles)+")"+ss+", "+ctype+", "+file)    
                try:
                    tree = ET.parse(inPath+file)
                    root = tree.getroot()
                    patterns=[]
                    for n in root:
                        print_path(n)

                    file_tokens = []
                    for p in patterns:
                        for t in p.split('-'):
                            if(len(t)>0):
                                file_tokens.append(t)

                    list_ts = []
                    for ngram in [2, 3]: #n-gram = 2 & 3
                        temp=zip(*[file_tokens[i:] for i in range(0, ngram)]) 
                        list_ts +=[' '.join(ngram) for ngram in temp]
                    
                    countedValues = Counter(list_ts)
                    
                    result_row = [file.split('.')[0]] #Take only the commit name from the file
                    ZeroLine = 0 #Checks if all the values in this row is zero
                    for lp in list_ts_features:
                        if(countedValues[lp] > 0):
                            ZeroLine += 1
                            result_row.append(countedValues[lp]) 
                        else: 
                            result_row.append(0)
                    
                    if(ZeroLine > 0):
                        dataset.append(result_row)

                except: not_worked += ss+", "+ctype+", "+file+"\n"
                    
            count+=1
                
#print(dataset)

Working for: (2604/2605)qtbase, added, ffef22cae224fd6c575e64a73e5017e5855c9aff.cpp.xml


In [16]:
pd.DataFrame(dataset).to_csv(ss+'_ts_dataset.csv', index=False, header=False)